# Sentence Segmentation - Rule-based methods

In [1]:
from sentence_segmentation_rules import rules
from utils import load, dump

In [2]:
file = load('data/pa153_playground_gold_all.txt')
dump('output/all_version_gold.txt', rules(file))

Loaded data/pa153_playground_gold_all.txt.
Written results to output/all_version_gold.txt


In [2]:
file = load('data/pa153_2025_test_all.txt')

Loaded data/pa153_2025_test_all.txt.


In [3]:
dump('output/all_version_2.txt', rules(file))

Written results to output/all_version_2.txt
